In [1]:
# import datasets
import pandas as pd

left_df = pd.read_csv("left_dataset.csv")
right_df = pd.read_csv("right_dataset.csv")

In [2]:
left_df.head()

,business_id,name,address,city,state,zip_code,size
0,1,SOURINI PAINTING INC.,12800 44th St N,Clearwater,FL,33762-4726,11.0
1,2,WOLFF DOLLA BILL LLC,1905 E 19th Ave,Tampa,FL,33605-2700,8.0
2,3,"COMPREHENSIVE SURGERY CENTER, LLC","1988 GULF TO BAY BLVD, Ste 1",CLEARWATER,FL,33765-3550,8.0
3,4,FRANK & ADAM APPAREL LLC,13640 Wright Cir,Tampa,FL,33626-3030,12.0
4,5,MORENO PLUS TRANSPORT INC,8608 Huron Court unite 58,Tampa,FL,33614,8.0


In [3]:
right_df.head()

,entity_id,name,address,city,state,postal_code,categories
0,1,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123.0,"Shipping Centers, Local Services, Notaries, Ma..."
1,2,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
2,3,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054.0,"Brewpubs, Breweries, Food"
3,4,Sonic Drive-In,615 S Main St,Ashland City,TN,37015.0,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
4,5,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144.0,"Sporting Goods, Fashion, Shoe Stores, Shopping..."


In [4]:
# get information about the datasets
# check the data types
print(left_df.dtypes)
print(right_df.dtypes)

business_id      int64
name            object
address         object
city            object
state           object
zip_code        object
size           float64
dtype: object
entity_id        int64
name            object
address         object
city            object
state           object
postal_code    float64
categories      object
dtype: object


In [5]:
# check the number of records and missing values
print(left_df.shape)
print(left_df.isnull().sum())

print(right_df.shape)
print(right_df.isnull().sum())

(98509, 7)
business_id    0
name           0
address        0
city           0
state          0
zip_code       0
size           0
dtype: int64
(94585, 7)
entity_id         0
name              0
address        2798
city              0
state             0
postal_code      37
categories       62
dtype: int64


In [6]:
# Address, posttal_code of the right data set have null value
null_address = right_df[right_df['address'].isnull()]
print(null_address.head())

     entity_id                               name address          city state  \
9           10              Vietnamese Food Truck     NaN     Tampa Bay    FL   
33          34                     Jennie Deckert     NaN        Carmel    IN   
57          58                            Impasto     NaN         Tampa    FL   
141        142               Mason's Tree Service     NaN  West Chester    PA   
155        156  Watson Adventures Scavenger Hunts     NaN  Philadelphia    PA   

     postal_code                                         categories  
9        33602.0         Vietnamese, Food, Restaurants, Food Trucks  
33       46032.0     Real Estate Agents, Home Services, Real Estate  
57       33611.0            Restaurants, Italian, Food Trucks, Food  
141      19380.0                       Home Services, Tree Services  
155      19019.0  Local Flavor, Team Building Activities, Active...  


In [19]:
# create copies for the datasets so that the original datasets can be kept the same for calculating confidence scores
left_dataset_2 = left_df.copy()
right_dataset_2 = right_df.copy()

In [20]:
#remove all the punctuation 
import re
punctuation = r'[^\w\s\']'

# Clean the "name", "address", and "city" columns in left dataset by removing punctuation
left_dataset_2['name'] = left_dataset_2['name'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())
left_dataset_2['address'] = left_dataset_2['address'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())
left_dataset_2['city'] = left_dataset_2['city'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())

# Clean the "name", "address", and "city" columns in right dataset by removing punctuation
right_dataset_2['name'] = right_dataset_2['name'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())
right_dataset_2['address'] = right_dataset_2['address'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())
right_dataset_2['city'] = right_dataset_2['city'].apply(lambda x: re.sub(punctuation, '', str(x)).lower())

In [21]:
# convert 'postal_code' in right_df to str
right_dataset_2['postal_code'] = right_dataset_2['postal_code'].astype(str)
right_dataset_2['postal_code'] = right_dataset_2['postal_code'].apply(lambda x: x.split('.')[0])

In [22]:
# strip the extra 4 digits of zip code
separator = "-"

for i in range(len(left_dataset_2)):
    if separator in left_dataset_2.loc[i, "zip_code"]:
        left_dataset_2.at[i,"zip_code"] = left_dataset_2.at[i,"zip_code"].split(separator)[0]

In [23]:
left_dataset_2.head()

,business_id,name,address,city,state,zip_code,size
0,1,sourini painting inc,12800 44th st n,clearwater,FL,33762,11.0
1,2,wolff dolla bill llc,1905 e 19th ave,tampa,FL,33605,8.0
2,3,comprehensive surgery center llc,1988 gulf to bay blvd ste 1,clearwater,FL,33765,8.0
3,4,frank adam apparel llc,13640 wright cir,tampa,FL,33626,12.0
4,5,moreno plus transport inc,8608 huron court unite 58,tampa,FL,33614,8.0


In [24]:
right_dataset_2.head()

,entity_id,name,address,city,state,postal_code,categories
0,1,the ups store,87 grasso plaza shopping center,affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma..."
1,2,st honore pastries,935 race st,philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
2,3,perkiomen valley brewery,101 walnut st,green lane,PA,18054,"Brewpubs, Breweries, Food"
3,4,sonic drivein,615 s main st,ashland city,TN,37015,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
4,5,famous footwear,8522 eager road dierbergs brentwood point,brentwood,MO,63144,"Sporting Goods, Fashion, Shoe Stores, Shopping..."


In [25]:
# create dictionary for abbreviations
# e, s, w, n, pkwy, hwy, st, apt...
# https://pe.usps.com/text/pub28/28apc_002.htm
'''
steps:
1. how to deal with null values? - need to 
2. create dictionary for abbreviations
- I think this is available https://pe.usps.com/text/pub28/28apc_002.htm

3. split strings into lists/dictionaries ex.:['uris','hall'] or {1:'uris',2:'hall'}
4. matching: zip code, state, city, street name, house number, name
5. score: length, list length (number of words)

SyntaxError: EOF while scanning triple-quoted string literal (1327471631.py, line 12)

In [27]:
# Create a dictionary of abbreviations and their corresponding expanded forms
abbrev_dict = {
    'aly': 'Alley',
    'ave': 'Avenue',
    'blvd': 'Boulevard',
    'byp': 'Bypass',
    'cir': 'Circle',
    'ct': 'Court',
    'dr': 'Drive',
    'expy': 'Expressway',
    'hwy': 'Highway',
    'ln': 'Lane',
    'pkwy': 'Parkway',
    'pl': 'Place',
    'pt': 'Point',
    'rd': 'Road',
    'sq': 'Square',
    'st': 'Street',
    'ter': 'Terrace',
    'trl': 'Trail',
    'ste':'Suite'
}
import re

# Define a function to replace abbreviations with their full forms
def replace_abbreviations(text):
    for abbrev, full in abbrev_dict.items():
        pattern = r"\b{}\b".format(re.escape(abbrev))
        text = re.sub(pattern, full, text, flags=re.IGNORECASE)
    return text

# Replace abbreviations in the "address" column of left_dataset_1
left_dataset_2['address'] = left_dataset_2['address'].apply(replace_abbreviations)

# Replace abbreviations in the "address" column of left_dataset_2
right_dataset_2['address'] = right_dataset_2['address'].apply(replace_abbreviations)



In [28]:
# change name, address, and city columns to lowercase and state column to uppercase
left_dataset_2['name'] = left_dataset_2['name'].str.lower()
left_dataset_2['address'] = left_dataset_2['address'].str.lower()
left_dataset_2['city'] = left_dataset_2['city'].str.lower()
left_dataset_2['state'] = left_dataset_2['state'].str.upper()

right_dataset_2['name'] = right_dataset_2['name'].str.lower()
right_dataset_2['address'] = right_dataset_2['address'].str.lower()
right_dataset_2['city'] = right_dataset_2['city'].str.lower()
right_dataset_2['state'] = right_dataset_2['state'].str.upper()

In [29]:
left_dataset_2.head()

,business_id,name,address,city,state,zip_code,size
0,1,sourini painting inc,12800 44th street n,clearwater,FL,33762,11.0
1,2,wolff dolla bill llc,1905 e 19th avenue,tampa,FL,33605,8.0
2,3,comprehensive surgery center llc,1988 gulf to bay boulevard suite 1,clearwater,FL,33765,8.0
3,4,frank adam apparel llc,13640 wright circle,tampa,FL,33626,12.0
4,5,moreno plus transport inc,8608 huron court unite 58,tampa,FL,33614,8.0


In [18]:
right_dataset_2.head()

,entity_id,name,address,city,state,postal_code,categories
0,1,the ups store,87 grasso placeaza shopping centerrace,affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma..."
1,2,st honore pastries,935 race street,philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
2,3,perkiomen valley brewery,101 walaneut street,green lane,PA,18054,"Brewpubs, Breweries, Food"
3,4,sonic drivein,615 s main street,ashland city,TN,37015,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
4,5,famous footwear,8522 eager road dierbergs brentwood point,brentwood,MO,63144,"Sporting Goods, Fashion, Shoe Stores, Shopping..."
